In [1]:
# === SEKCJA 1: IMPORT I KONFIGURACJA ===
import pandas as pd
import numpy as np
import mlflow
import re
from pycaret.classification import setup, create_model, tune_model, finalize_model, save_model, pull
from IPython.display import display

MLFLOW_EXPERIMENT_NAME = 'Warsaw_Street_Prediction_v6_STABLE'
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

print("Wczytywanie danych...")
df_main = pd.read_csv('data.csv', sep=',')
df_ulic = pd.read_csv('Ulic.csv', sep=',')
print("Wszystkie pliki wczytane.")

Wczytywanie danych...
Wszystkie pliki wczytane.


In [2]:
# === SEKCJA 2: PEŁNE PRZYGOTOWANIE I WZBOGACENIE DANYCH ===

# --- Krok 2a: Stworzenie słownika ulic z pliku Ulic.csv ---
print("Krok 2a: Tworzenie słownika ulic...")
# Poprawne nazwy kolumn z Twojego pliku: 'SymUl' i 'Name'
df_ulic.rename(columns={'Name': 'NAZWA_ULICY'}, inplace=True)
df_ulic.dropna(subset=['SymUl', 'NAZWA_ULICY'], inplace=True)
street_dictionary = pd.Series(df_ulic.NAZWA_ULICY.values, index=df_ulic.SymUl).to_dict()
print(f"Utworzono słownik z {len(street_dictionary)} unikalnymi ulicami.")


# --- Krok 2b: Czyszczenie i inżynieria cech ---
print("\nKrok 2b: Rozpoczynam czyszczenie i wzbogacanie danych...")
df_processed = df_main.copy()
df_processed.dropna(subset=['Area', 'Price', 'Description', 'Location'], inplace=True)
print(f"Rozmiar po usunięciu podstawowych NaN: {df_processed.shape}")

# Definicja funkcji wewnątrz tego samego bloku, aby miała dostęp do `street_dictionary`
def process_location(row):
    location_str = row['Location']
    district, street = np.nan, np.nan
    
    if isinstance(location_str, str):
        parts = [p.strip() for p in location_str.split(',')]
        if len(parts) >= 3: district = parts[2]
    
    # Użyj StreetNumber do odnalezienia w słowniku
    if pd.notna(row['StreetNumber']):
        try:
            street_sym = int(row['StreetNumber'])
            if street_sym in street_dictionary:
                street = street_dictionary[street_sym]
        except (ValueError, TypeError): pass
            
    # Jeśli słownik zawiódł, próbuj parsować tekst
    if pd.isna(street) and isinstance(location_str, str) and len(location_str.split(',')) >= 4:
        street = location_str.split(',')[3].strip()

    # Czyszczenie nazwy ulicy
    if isinstance(street, str):
        street = re.sub(r'^(ul\.|al\.|pl\.)\s*', '', street, flags=re.IGNORECASE).lower()
        if len(street) < 3: street = np.nan
            
    return pd.Series([district, street])

# Zastosowanie funkcji
df_processed[['District', 'Ulica_clean']] = df_processed.apply(process_location, axis=1)
df_processed.dropna(subset=['Ulica_clean'], inplace=True)
print(f"Po ekstrakcji ulic -> Pozostało wierszy: {len(df_processed)}, Unikalnych ulic: {df_processed['Ulica_clean'].nunique()}")

# --- Krok 2c: Finalne czyszczenie ---
df_processed['BuiltYear'] = pd.to_datetime(df_processed['BuiltYear'], format='%Y', errors='coerce')
q_low, q_hi = df_processed["Price"].quantile(0.01), df_processed["Price"].quantile(0.99)
df_final = df_processed[(df_processed["Price"] < q_hi) & (df_processed["Price"] > q_low)].copy()
print(f"Po usunięciu outlierów cenowych -> Pozostało wierszy: {len(df_final)}")

Krok 2a: Tworzenie słownika ulic...
Utworzono słownik z 47823 unikalnymi ulicami.

Krok 2b: Rozpoczynam czyszczenie i wzbogacanie danych...
Rozmiar po usunięciu podstawowych NaN: (181622, 51)
Po ekstrakcji ulic -> Pozostało wierszy: 111833, Unikalnych ulic: 4201
Po usunięciu outlierów cenowych -> Pozostało wierszy: 109562


In [3]:
# === SEKCJA 3: PRZYGOTOWANIE DO MODELOWANIA I SETUP (WERSJA STRATYFIKOWANA) ===
from sklearn.model_selection import train_test_split

# --- Krok 3a: Filtrowanie rzadkich ULIC ---
MIN_SAMPLES_PER_STREET = 25 # Zostawiamy próg, który dawał ~1000 klas
street_counts = df_final['Ulica_clean'].value_counts()
streets_to_remove = street_counts[street_counts < MIN_SAMPLES_PER_STREET].index
df_model_ready = df_final[~df_final['Ulica_clean'].isin(streets_to_remove)].copy()
print(f"Po odfiltrowaniu rzadkich ulic -> Pozostało wierszy: {len(df_model_ready)}, Unikalnych ulic: {df_model_ready['Ulica_clean'].nunique()}")

# --- Krok 3b (NAJWAŻNIEJSZA ZMIANA): Ręczny podział stratyfikowany ---
# Dzielimy dane PRZED setup, gwarantując, że proporcje ulic są zachowane w obu zbiorach.
train_data, test_data = train_test_split(
    df_model_ready, 
    test_size=0.1, 
    random_state=1122, 
    stratify=df_model_ready['Ulica_clean'] # To jest klucz do sukcesu
)
print(f"\nPodział stratyfikowany -> Treningowe: {train_data.shape}, Testowe: {test_data.shape}")

# --- Krok 3c: Setup PyCaret ---
# Używamy domyślnej, stabilnej strategii 'stratifiedkfold'.
s = setup(
    data=train_data,
    test_data=test_data,
    target='Ulica_clean',
    session_id=1122,
    n_jobs=4,
    
    # Używamy standardowej, stabilnej strategii walidacji
    fold_strategy='stratifiedkfold', 
    
    # Ignorujemy kolumny, które nie są cechami lub spowodowałyby wyciek danych
    ignore_features=['SaleId', 'OriginalId', 'Location', 'Description', 'Title', 'Link', 'Phone', 'MainImage', 'OtherImages', 'EncryptedId', 'NAZWA_ULICY'],
    
    log_experiment=False,
    html=False,
    verbose=True
)

Po odfiltrowaniu rzadkich ulic -> Pozostało wierszy: 88697, Unikalnych ulic: 1000

Podział stratyfikowany -> Treningowe: (79827, 53), Testowe: (8870, 53)
                    Description  \
0                    Session id   
1                        Target   
2                   Target type   
3                Target mapping   
4           Original data shape   
5        Transformed data shape   
6   Transformed train set shape   
7    Transformed test set shape   
8               Ignore features   
9              Numeric features   
10                Date features   
11         Categorical features   
12     Rows with missing values   
13                   Preprocess   
14              Imputation type   
15           Numeric imputation   
16       Categorical imputation   
17     Maximum one-hot encoding   
18              Encoding method   
19               Fold Generator   
20                  Fold Number   
21                     CPU Jobs   
22                      Use GPU   
23    

In [4]:
# === SEKCJA 4: KONTROLOWANY TRENING JEDNEGO MODELU ===
import traceback

print("\nRozpoczynam kontrolowany trening modelu 'lightgbm'...")
lgbm_model = None
try:
    # Używamy create_model. `sort` jest niepotrzebny, bo to nie jest porównanie.
    lgbm_model = create_model('lightgbm', verbose=True)
    
    # Jeśli ten kod się wykona, to znaczy, że model się wytrenował
    print("\n" + "="*50)
    print("SUKCES! Model 'lightgbm' został pomyślnie wytrenowany.")
    print("Metryki dla 'lightgbm':")
    display(pull())
    
except Exception as e:
    # Jeśli kod się wysypie, w końcu zobaczymy prawdziwy błąd
    print("\n" + "="*50)
    print(f"BŁĄD KRYTYCZNY PODCZAS TWORZENIA MODELU:")
    print(f"Typ błędu: {type(e).__name__}")
    print(f"Treść błędu: {e}")
    print("\nPełny traceback:")
    traceback.print_exc()
    print("="*50)


Rozpoczynam kontrolowany trening modelu 'lightgbm'...


      Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
Fold                                                          
0       0.0457  0.5245  0.0457  0.0274  0.0224  0.0430  0.0446
1       0.0089  0.5005  0.0089  0.0001  0.0003  0.0010  0.0014
2       0.0418  0.5242  0.0418  0.0323  0.0249  0.0389  0.0398
3       0.0448  0.5304  0.0448  0.0342  0.0304  0.0429  0.0432
4       0.0355  0.5219  0.0355  0.0371  0.0249  0.0338  0.0344
5       0.0091  0.5005  0.0091  0.0065  0.0015  0.0010  0.0037
6       0.0157  0.5065  0.0157  0.0124  0.0095  0.0118  0.0187
7       0.0069  0.5001  0.0069  0.0033  0.0006  0.0002  0.0011
8       0.0119  0.5001  0.0119  0.0135  0.0008  0.0002  0.0015
9       0.0064  0.5012  0.0064  0.0030  0.0022  0.0020  0.0079
Mean    0.0227  0.5110  0.0227  0.0170  0.0117  0.0175  0.0196
Std     0.0161  0.0119  0.0161  0.0136  0.0118  0.0185  0.0179

SUKCES! Model 'lightgbm' został pomyślnie wytrenowany.
Metryki dla 'lightgbm':


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.0457,0.5245,0.0457,0.0274,0.0224,0.0430,0.0446
1,0.0089,0.5005,0.0089,0.0001,0.0003,0.0010,0.0014
2,0.0418,0.5242,0.0418,0.0323,0.0249,0.0389,0.0398
3,0.0448,0.5304,0.0448,0.0342,0.0304,0.0429,0.0432
4,0.0355,0.5219,0.0355,0.0371,0.0249,0.0338,0.0344
5,0.0091,0.5005,0.0091,0.0065,0.0015,0.0010,0.0037
6,0.0157,0.5065,0.0157,0.0124,0.0095,0.0118,0.0187
7,0.0069,0.5001,0.0069,0.0033,0.0006,0.0002,0.0011
8,0.0119,0.5001,0.0119,0.0135,0.0008,0.0002,0.0015
